In [1]:
"""
Will generate the skeletons for all of the
Neurites

Process: 
1) Pull from ta3p100.Mesh
-- DO NOT NEED TO FILTER B/C DON'T HAVE LABELS
2) Voxelize the mesh 
3) Export the mesh to an off file
4) Clean up the mesh using:
a. smoothing normals
b. Poisson surface reconstruction
c. Extra remove duplicate filters
5) Send through skeletonization
6) Reading in skeleton segments
7) Write key to database

"""



"\nWill generate the skeletons for all of the\nNeurites\n\nProcess: \n1) Pull from ta3p100.Mesh\n-- DO NOT NEED TO FILTER B/C DON'T HAVE LABELS\n2) Voxelize the mesh \n3) Export the mesh to an off file\n4) Clean up the mesh using:\na. smoothing normals\nb. Poisson surface reconstruction\nc. Extra remove duplicate filters\n5) Send through skeletonization\n6) Reading in skeleton segments\n7) Write key to database\n\n"

In [2]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [4]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)




In [5]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/temp_meshlab")) == False:
    os.mkdir("temp_meshlab")
    
    

In [6]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [7]:
#get keysource:
ta3p100.Mesh() & ta3p100.Neurite() #32207 tuples

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
2,648518346341351467,28754,57350,=BLOB=,=BLOB=
2,648518346341351503,11192,22282,=BLOB=,=BLOB=
2,648518346341351508,58449,116104,=BLOB=,=BLOB=
2,648518346341351509,24919,49578,=BLOB=,=BLOB=
2,648518346341351512,37179,74128,=BLOB=,=BLOB=
2,648518346341351514,37788,75374,=BLOB=,=BLOB=
2,648518346341351518,20204,40278,=BLOB=,=BLOB=
2,648518346341351523,15522,30788,=BLOB=,=BLOB=
2,648518346341351525,45964,91650,=BLOB=,=BLOB=
2,648518346341351531,21520,42882,=BLOB=,=BLOB=


In [8]:
def meshlab_shrinkwrap(key):
    
    file_loc = pathlib.Path.cwd() / "temp_meshlab"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    midoutput_mesh = path_and_filename + "_mid.off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_script.mlx"
    meshlab_script_rem_dupl = str(pathlib.Path.cwd()) + "/" + "remeshing_script-Remove.mlx"
    #send to meshlabserver
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      midoutput_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not get pass Poisson')
    #print(type)
     
    
    #do another call to remove the final duplicate vertices
    print("starting meshlabserver cleaning")
    subprocess_result_2 = run_meshlab_script(meshlab_script_rem_dupl,
                      midoutput_mesh,
                      output_mesh)
    #print("Cleaning subprocess_result= "+ str(subprocess_result_2))
    
    if str(subprocess_result_2)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not get pass cleaning')
    
    
    return output_mesh

In [9]:
#don't need the vertices for this one because not doing any filtering

@schema
class NeuriteSkeletons(dj.Computed):
    definition="""
    -> ta3p100.Mesh
    time_updated      :timestamp    # the time at which the skeleton was generated
    ---
    n_branches   :int unsigned #number of edges stored
    branches     :longblob #array storing vertices of edges and each seperated by Nan
    """
    
    key_source = ta3p100.Mesh() & ta3p100.Neurite() 
    
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
        print("Starting on "+str(key["segment_id"]))
        global_time = time.time()
        #get the mesh with the error segments filtered away
        start_time = time.time()
        
        file_loc = pathlib.Path.cwd() / "temp_meshlab"
        filename = "neuron_" + str(key["segment_id"])
        path_and_filename = str(file_loc / filename)
        
        #get the mesh of the neurite
        new_key = dict(segmentation=key["segmentation"],
                       segment_id=key["segment_id"])
        mesh = (ta3p100.Mesh & new_key).fetch1()
        print(f"Step 1: Retrieving Mesh: {time.time() - start_time}")
        
        #Do voxelization
        start_time = time.time()
        mesh = trimesh_io.Mesh(vertices=mesh["vertices"], faces=mesh["triangles"])
        voxels = mesh.voxelized(500)
        voxel_mesh = voxels.as_boxes()
        print(f"Step 2: Voxelization: {time.time() - start_time}")
        
        #Exporting the Voxelization as an off file for meshlabserver
        start_time = time.time()
        #try the inline printing method:
        with open(os.devnull, 'w') as devnull:
            with contextlib.redirect_stdout(devnull):
                voxel_mesh.export(str(path_and_filename) + ".off")
        print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")
        
        #Run the meshlabserver scripts
        start_time = time.time()
        output_mesh = meshlab_shrinkwrap(key)
        print(f"Step 4: Meshlab shrinkwrapping: {time.time() - start_time}")
        
        #Create the skeleton and retrieve it from the generated file
        #skeletonize the mesh
        start_time = time.time()
        #print("starting creating skeleton")
        return_value = cm.calcification(output_mesh[:-4])
        #print("calcif_return_value = " + str(return_value))
        
        if return_value > 0:
            raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                            ' did not finish... exited with error code: ' + str(return_value))
        print(f"Step 4: Generating Skeleton: {time.time() - start_time}")
        
        #read in the skeleton files into an array
        start_time = time.time()
        bone_array = read_skeleton_flat(output_mesh[:-4]+"_skeleton.cgal")
        #print(bone_array)
        if len(bone_array) <= 0:
            raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
        print(f"Step 5: Reading in Skeleton: {time.time() - start_time}")
        
              
        start_time = time.time()
        new_key["n_branches"] = len(bone_array)
        new_key["branches"] = bone_array
        #new_key["branches"] = []
        
        
        new_key["time_updated"]=str(datetime.datetime.now())
        #print(new_key)
        #if all goes well then write to database
        self.insert1(new_key)
        os.system("rm "+str(path_and_filename)+"*")
        print(f"Step 6: Inserting dictionary and erased files: {time.time() - start_time}")
        print(f"Total time: {time.time() - global_time}")
        print("\n\n")

In [10]:
start = time.time()
NeuriteSkeletons.populate(reserve_jobs=True)
print(time.time() - start)

Starting on 648518346341352911
Step 1: Retrieving Mesh: 0.04831814765930176
Step 2: Voxelization: 0.09458017349243164
Step 3: Exporting Voxel Off function: 0.03656649589538574
starting meshlabserver Poisson surface reconstruction
starting meshlabserver cleaning
Step 4: Meshlab shrinkwrapping: 4.443084001541138
Step 4: Generating Skeleton: 0.3896353244781494
Step 5: Reading in Skeleton: 0.0014810562133789062
Step 6: Inserting dictionary and erased files: 0.02983260154724121
Total time: 5.046126127243042



Starting on 648518346341353060
Step 1: Retrieving Mesh: 0.03585529327392578
Step 2: Voxelization: 0.10836505889892578
Step 3: Exporting Voxel Off function: 0.05378580093383789
starting meshlabserver Poisson surface reconstruction
starting meshlabserver cleaning
Step 4: Meshlab shrinkwrapping: 4.603152513504028
Step 4: Generating Skeleton: 0.45560598373413086
Step 5: Reading in Skeleton: 0.002617359161376953
Step 6: Inserting dictionary and erased files: 0.040997982025146484
Total time

Exception: skeletonization for neuron 648518346341357003 did not finish... exited with error code: 4

In [ ]:
ta3p100.NeuriteSkeletons().delete()